Анализ тональности к именованным сущностям в новостных текстах

[Репозиторий на GitHub](https://github.com/dialogue-evaluation/RuSentNE-evaluation)

[Страница на CodaLab](https://codalab.lisn.upsaclay.fr/competitions/9538)



## Данные

### Загрузка

Загрузим обучающую, валидационную и тестовую выборки.

In [ ]:
!wget -q https://raw.githubusercontent.com/dialogue-evaluation/RuSentNE-evaluation/main/train_data.csv
!wget -q https://raw.githubusercontent.com/dialogue-evaluation/RuSentNE-evaluation/main/validation_data_labeled.csv
!wget -q https://raw.githubusercontent.com/dialogue-evaluation/RuSentNE-evaluation/main/final_data.csv

In [ ]:
import pandas as pd
train = pd.read_csv('train_data.csv', sep='\t')
print(train.shape)
train.head()

In [ ]:
validation = pd.read_csv('validation_data_labeled.csv', sep='\t')
print(validation.shape)
validation.head()

In [ ]:
test = pd.read_csv('final_data.csv', sep='\t')
print(test.shape)
test.head()

Разметка присутствует в обучающей и валидационной выборке. Разметка для тестовых данных отсутствует в загруженном файле. Оценка качестве на тестовой выборке возможна только через платформу CodaLab.

### Анализ

Проанализируем данные обучающей выборки.

Определим минимальную, максимальную и среднюю длину текста. Отобразим распределение на графике.

In [ ]:
lens = [len(x.split()) for x in train['sentence']]

max_l, min_l, mean_l = max(lens), min(lens), sum(lens)/len(lens)

print(f'Минимальная длина текста: {min_l}')
print(f'Максимальная длина текста: {max_l}')
print(f'Средняя длина текста: {mean_l:.3f}')

In [ ]:
from collections import Counter
from matplotlib import pyplot as plt

len_counts = Counter(lens)
plt.figure(figsize = (6,3))
plt.bar(len_counts.keys(), len_counts.values())

Выведем самый длинный текст.

In [ ]:
for i in range(len(train)):
    if len(train['sentence'][i].split()) == max_l:
        print(f"Предложение:\n{train['sentence'][i]}")
        print(f"Сущность:\n{train['entity'][i]}")
        print(f"Тональность:\n{train['label'][i]}\n")

Посмотрим на распределение классов тональности.

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.xlabel('Класс')
plt.ylabel('Количество примеров')
plt.title('Распределение примеров по классам')
plt.bar(1, train[train['label'] == 1].shape, label='POS', color='lightgreen')
plt.bar(0, train[train['label'] == 0].shape, label='NEUT', color='lightblue')
plt.bar(-1, train[train['label'] == -1].shape, label='NEG', color='lightpink')
plt.xticks(ticks=[1, 0, -1], labels=['1', '0', '-1'])
plt.legend()
plt.show()

Посмотрим статистику по типам сущностей.

In [ ]:
plt.figure()
plt.xlabel('Сущность')
plt.ylabel('Количество примеров')
plt.title('Распределение типов сущностей')
plt.bar('Люди', train[train['entity_tag'] == 'PERSON'].shape)
plt.bar('Профессии', train[train['entity_tag'] == 'PROFESSION'].shape)
plt.bar('Организации', train[train['entity_tag'] == 'ORGANIZATION'].shape)
plt.bar('Страны', train[train['entity_tag'] == 'COUNTRY'].shape)
plt.bar('Национальности', train[train['entity_tag'] == 'NATIONALITY'].shape)
plt.xticks(ticks=['Люди', 'Профессии', 'Организации', 'Страны', 'Национальности'])
plt.tick_params(labelrotation=15)
plt.show()

Определим самые частые сущности.

In [ ]:
train['entity'].value_counts()

## Модель

В этом разделе осуществим тонкую настройку модели [RuBERT base conversational](https://huggingface.co/DeepPavlov/rubert-base-cased-conversational).

### Создание вопросов

Будем решать задачу классификации пары предложений. На вход подаются два предложения, разделенные токеном [SEP]:
1. Вопрос «Как относятся к X?» где Х – сущность в дательном падеже;
2. Текст предложения.

Следовательно, необходимо каждое предложение сопроводить вопросом. Для постановки сущности в дательный падеж воспользуемся библиотекой [pymorphy3](https://github.com/no-plagiarism/pymorphy3). Документация аналогична предыдущей версии библиотеки [pymorphy2](https://pymorphy2.readthedocs.io/en/stable/user/guide.html).

In [ ]:
!pip install -q pymorphy3

In [ ]:
import pymorphy3
morph = pymorphy3.MorphAnalyzer()

С помощью метода `morph.parse()` получаем все возможные морфологические разборы слова и берем первый вариант.

In [ ]:
word = morph.parse(train['entity'][0])
word

 С помощью метода `.inflect()` ставим слово в нужный падеж.

In [ ]:
dative = word[0].inflect({'datv'})
dative

 С помощью метода `.restore_capitalization()` сохраняем исходный регистр.  Применяем это к каждому слову, входящему в название сущности.

In [ ]:
def question(df):
  sentences = []
  for entity in df['entity'].values:
    try:
      dative_list = [pymorphy3.shapes.restore_capitalization(morph.parse(x)[0].inflect({'datv'}).word, x) for x in entity.split()]
      final_form = ' '.join(dative_list)
    except AttributeError:
      final_form = entity
    sentences.append(f'Как относятся к {final_form}?')
  return sentences

Добавим вопросы для объектов обучающей, валидационной и тестовой выборки.

In [ ]:
train['question'] = question(train)
train.head()

In [ ]:
validation['question'] = question(validation)
validation.head()

In [ ]:
test['question'] = question(test)
test.head()

### Предобработка

Исходно метки датасета выглядят так: -1 - отрицательная тональность, 0 - нейтральная, 1 - положительная.

При обучении нейросети мы выбираем предсказанный класс как позицию наибольшего значения (наибольшей вероятности). Следовательно, необходимо избавиться от отрицательных значений в метках класса.

Применим преобразование к обучающей и валидационной выборке.

In [ ]:
label_dict = {-1: 0, 0: 1, 1: 2}
train['raw_label'] = train["label"]
train['label'] = train["raw_label"].map(label_dict)
train.head()

In [ ]:
validation['raw_label'] = validation["label"]
validation['label'] = validation["raw_label"].map(label_dict)
validation.head()

Преобразуем набор данных в датасет Hugging Face.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install -q datasets transformers evaluate

In [ ]:
from datasets import Dataset, DatasetDict
dataset_dict = DatasetDict({"train": Dataset.from_pandas(train),
                            "validation": Dataset.from_pandas(validation),
                            "test": Dataset.from_pandas(test)})

Мы получили объект класса `DatasetDict`, который включает обучающую выборку, валидационную выборку и тестовую выборку.

In [ ]:
dataset_dict

In [ ]:
dataset_dict["train"][0]

Чтобы предобработать датасет, нам необходимо конвертировать текст в числа, которые может обработать модель. Это делается с помощью токенизатора.

In [ ]:
from transformers import AutoTokenizer

checkpoint = "DeepPavlov/rubert-base-cased-conversational"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Чтобы хранить данные в формате датасета, мы будем использовать методы `Dataset.map()`. Метод `map()` применяет некоторую функцию к каждому элементу датасета.

Давайте определим функцию, которая токенизирует наши входные данные:

In [ ]:
def tokenize_function(example):
    return tokenizer(example["question"], example["sentence"])

In [ ]:
tokenized_dataset = dataset_dict.map(tokenize_function, batched=True)
tokenized_dataset

`input_ids` содержит индексы, соответствующие токенам по словарю.

Маски внимания (`attention_mask`) — это тензоры той же формы, что и тензор входных идентификаторов, заполненные 0 и 1: 1 означает, что соответствующие токены должны “привлекать внимание”, а 0 означает, что соответствующие токены не должны “привлекать внимание” (т.е. должны игнорироваться слоями внимания модели).

`token_type_ids` указывает модели, какая часть входных данных является первым предложением, а какая вторым.

In [ ]:
print(f'input_ids\n{tokenized_dataset["train"][0]["input_ids"]}')
print(f'attention_mask\n{tokenized_dataset["train"][0]["attention_mask"]}')
print(f'token_type_ids\n{tokenized_dataset["train"][0]["token_type_ids"]}')

Функция, отвечающая за объединение элементов внутри батча, называется `collate_function`. Это аргумент, который вы можете передать при создании `DataLoader`. По умолчанию это функция, которая просто преобразует объекты в тензоры PyTorch и объединяет их. В нашем случае это невозможно, поскольку входные данные, которые у нас есть, не будут иметь одинакового размера.

Функция `collate_function` будет осуществлять корректный паддинг элементов выборки, которые мы хотим объединить в батч. Библиотека Transformers предоставляет эту функцию через класс `DataCollatorWithPadding`. При создании экземпляра требуется указать токенизатор: чтобы знать, какой токен использовать для паддинга и слева или справа нужно дополнять данные.

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Мы хотим использовать GPU в случае, если у нас будет такая возможность (на CPU процесс может занять несколько часов вместо пары минут). Чтобы добиться этого, мы определим переменную `device` и «прикрепим» к видеокарте нашу модель и данные.

In [ ]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

Перейдем к тонкой настройке.

### Trainer

Библиотека Transformers предоставляет класс `Trainer`, который помогает произвести тонкую настройку любой предобученной модели на вашем датасете. После предобработки данных, сделанных в прошлом разделе, вам останется сделать несколько шагов для определения `Trainer`.

Первый шаг перед определением `Trainer` — задание класса `TrainingArguments`, который будет содержать все гиперпараметры для `Trainer` (для процессов обучения и валидации). Единственный аргумент, который обязательно нужно задать, — это каталог, в котором будет сохранена обученная модель. Для всего остального можно оставить значения по умолчанию.

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir='./results',
                                  evaluation_strategy="epoch",
                                  report_to="none")

Второй шаг – задание модели. Мы будем использовать класс `AutoModelForSequenceClassification` с тремя классами:

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3).to(device)

После того как мы загрузили модель, мы можем определить `Trainer` и передать туда нужные объекты: `model`, `training_args`, обучающую и валидационную выборки, `data_collator` и `tokenizer`.

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    processing_class=tokenizer
)

In [ ]:
trainer.model

Для тонкой настройки модели на нашем датасете нужно вызвать метод `train()` у `Trainer`:

In [ ]:
trainer.train()

Чтобы получить предсказания, мы можем использовать встроенную функцию `Trainer.predict()`. В качестве выходов модели получаем различные данные, нам нужны именно предсказания и истинные метки. Чтобы определить предсказанный класс, будем брать позицию максимального значения (`argmax`) по строке (`axis=-1`). Также осуществим обратное преобразование меток. Объединим все этапы в функцию `predict_labels`.

In [ ]:
import numpy as np

def predict_labels(dataset):
    output = trainer.predict(dataset)
    logits, labels = output[:2]
    predictions = np.argmax(logits, axis=-1)
    reverse_label_dict = {v:k for k, v in label_dict.items()}
    return [reverse_label_dict[x] for x in predictions]

Запишем предсказания на валидационной выборке и добавим их в датасет.

In [ ]:
validation_predictions = predict_labels(tokenized_dataset["validation"])
print(len(validation_predictions))
validation_predictions[:25]

In [ ]:
tokenized_dataset["validation"] = tokenized_dataset["validation"].add_column("predictions", validation_predictions)
tokenized_dataset

Посчитаем значение макро F1-меры.

In [ ]:
import evaluate

def compute_metrics(preds, labels):
    metric = evaluate.load("f1")
    return metric.compute(predictions=preds, references=labels, average="macro")

In [ ]:
compute_metrics(tokenized_dataset["validation"]["predictions"], tokenized_dataset["validation"]["raw_label"])

Однако в соревновании рейтинг строится по макро F1-мере, посчитанной только для положительного и отрицательного класса. Уберем нейтральный класс и посчитаем метрику.

In [ ]:
filtered_validation = tokenized_dataset["validation"].filter(lambda example: example["raw_label"]!=0)
filtered_validation

In [ ]:
compute_metrics(filtered_validation["predictions"], filtered_validation["raw_label"])

Осуществим предсказания на тестовой выборке и запишем их в файл.

In [ ]:
test_predictions = predict_labels(tokenized_dataset["test"])
print(len(test_predictions))
test_predictions[1925:]

In [ ]:
pd.Series(test_predictions).to_csv('RuSentNE_predictions_Trainer.zip', compression={'method': 'zip', 'archive_name': 'RuSentNE_predictions_Trainer.csv'}, index=False, header=False)

Итоговый zip-архив, содержащий внутри себя csv-файл, может быть отправлен на платформу CodaLab для получения метрики на тестовой выборке.